In [12]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPool2D , Flatten, Dropout, GlobalAveragePooling2D, AveragePooling2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

In [13]:

tf.config.list_physical_devices('GPU')
#print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [14]:
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

GPUs:  1


In [15]:
train = pd.read_csv('train_metadata.csv')
test = pd.read_csv('test_metadata.csv')
#submission = pd.read_csv('sample_submission.csv')

train_dir = 'train/'
test_dir = 'test/'

In [16]:
label = []
train_data = []
for i in range(train.shape[0]):
    train_data.append(train_dir + train['image_name'].iloc[i] + '.jpg')
    label.append(train['diagnosis'].iloc[i])
    
test_data = []
for i in range(test.shape[0]):
    test_data.append(test_dir + test['image'].iloc[i] + '.jpg')

In [17]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
label_df = pd.DataFrame(label, columns = ['diagnosis'])
one_hot_encoder.fit(label_df)

label_df_encoded = one_hot_encoder.transform(label_df)
label_df_encoded = pd.DataFrame(data=label_df_encoded, columns=one_hot_encoder.categories_)
print(label_df_encoded.columns)

MultiIndex([('atypical melanocytic proliferation',),
            (               'cafe-au-lait macule',),
            (                       'lentigo NOS',),
            (               'lichenoid keratosis',),
            (                          'melanoma',),
            (                             'nevus',),
            (              'seborrheic keratosis',),
            (                     'solar lentigo',),
            (                           'unknown',)],
           )


In [18]:
df_train = pd.DataFrame(train_data)
df_train.columns = ['images']
df_train = pd.concat([df_train, label_df_encoded], axis = 1)
print(df_train.columns)
df_test = pd.DataFrame(test_data)
df_test.columns = ['images']

Index([                               'images',
       ('atypical melanocytic proliferation',),
                      ('cafe-au-lait macule',),
                              ('lentigo NOS',),
                      ('lichenoid keratosis',),
                                 ('melanoma',),
                                    ('nevus',),
                     ('seborrheic keratosis',),
                            ('solar lentigo',),
                                  ('unknown',)],
      dtype='object')


In [19]:
X_train, X_test, y_train, y_test = train_test_split(df_train.iloc[:, 0], df_train.iloc[:,1::], test_size = 0.3, random_state = 42)

train = pd.DataFrame(X_train)
train.columns = ['images']
train = pd.concat([train, y_train], axis = 1)

test = pd.DataFrame(X_test)
test.columns = ['images']
test = pd.concat([test, y_test], axis = 1)

In [20]:
train.columns[1:]

Index([('atypical melanocytic proliferation',),
                      ('cafe-au-lait macule',),
                              ('lentigo NOS',),
                      ('lichenoid keratosis',),
                                 ('melanoma',),
                                    ('nevus',),
                     ('seborrheic keratosis',),
                            ('solar lentigo',),
                                  ('unknown',)],
      dtype='object')

In [21]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=50,
                                   zoom_range = 0.3,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range = 0.2,
                                   horizontal_flip=True,
                                   fill_mode = 'nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col=train.columns[0],
    y_col=train.columns[1:],
    target_size=(224, 224),
    batch_size=8,
    shuffle=True,
    class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(
    test,
    x_col=test.columns[0],
    y_col=test.columns[1:],
    target_size=(224, 224),
    shuffle=False,
    batch_size=8,
    class_mode='raw')

Found 23188 validated image filenames.
Found 9938 validated image filenames.


In [22]:
#Evaluate
"""model = keras.models.load_model("Models/vgg16_1.h5")
scores = model.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))

scores = model.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))"""

'model = keras.models.load_model("Models/vgg16_1.h5")\nscores = model.evaluate(test_generator)\nprint("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))\n\nscores = model.evaluate_generator(test_generator)\nprint("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))'

In [23]:
#Evaluate
model = keras.models.load_model("Models/vgg19.h5")
scores = model.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))

scores = model.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))

1243/1243 [==============================] - 345s 278ms/step - loss: 0.3043 - accuracy: 0.9417
evaluate accuracy: 94.17%


/home/deeplab/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


evaluate_generator accuracy: 94.17%


In [24]:
#Evaluate
model = keras.models.load_model("Models/dense-169.h5")
scores = model.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))

scores = model.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))

1243/1243 [==============================] - 335s 268ms/step - loss: 0.2730 - accuracy: 0.9288
evaluate accuracy: 92.88%
evaluate_generator accuracy: 92.88%


In [25]:
#Evaluate
model = keras.models.load_model("Models/dense-201.h5")
scores = model.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))

scores = model.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))

1243/1243 [==============================] - 346s 277ms/step - loss: 0.3105 - accuracy: 0.9175
evaluate accuracy: 91.75%
evaluate_generator accuracy: 91.75%


In [26]:
#Evaluate
model = keras.models.load_model("Models/resnet-50.h5")
scores = model.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))

scores = model.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))

1243/1243 [==============================] - 323s 260ms/step - loss: 0.2334 - accuracy: 0.9492
evaluate accuracy: 94.92%
evaluate_generator accuracy: 94.92%


In [27]:
#Evaluate
model = keras.models.load_model("Models/resnet-152.h5")
scores = model.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))

scores = model.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))

1243/1243 [==============================] - 321s 257ms/step - loss: 0.2857 - accuracy: 0.9463
evaluate accuracy: 94.63%
evaluate_generator accuracy: 94.63%


In [28]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=50,
                                   zoom_range = 0.3,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range = 0.2,
                                   horizontal_flip=True,
                                   fill_mode = 'nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col=train.columns[0],
    y_col=train.columns[1:],
    target_size=(300, 300),
    batch_size=8,
    shuffle=True,
    class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(
    test,
    x_col=test.columns[0],
    y_col=test.columns[1:],
    target_size=(300, 300),
    shuffle=False,
    batch_size=8,
    class_mode='raw')

Found 23188 validated image filenames.
Found 9938 validated image filenames.


In [29]:
#Evaluate
model = keras.models.load_model("Models/efficientB3_1.h5")
scores = model.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))

scores = model.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))

1243/1243 [==============================] - 327s 262ms/step - loss: 0.2395 - accuracy: 0.9415
evaluate accuracy: 94.15%
evaluate_generator accuracy: 94.15%
